# Wind Data Combiner

Este notebook combina os dados de três arquivos de vento (Dir.txt, W.txt, WS.txt) em um único dataset regularizado.

In [1]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("Bibliotecas importadas com sucesso")

Bibliotecas importadas com sucesso


In [2]:
# Definir caminhos dos arquivos
raw_data_path = Path('../raw_data')
output_path = Path('../')

# Lista dos arquivos a serem processados
files_to_process = ['Dir.txt', 'W.txt', 'WS.txt']

print(f"Diretório de dados brutos: {raw_data_path}")
print(f"Diretório de saída: {output_path}")
print(f"Arquivos a processar: {files_to_process}")

Diretório de dados brutos: ../raw_data
Diretório de saída: ..
Arquivos a processar: ['Dir.txt', 'W.txt', 'WS.txt']


In [3]:
# Função para ler e processar cada arquivo
def read_wind_data(file_path, prefix):
    """
    Lê arquivo de dados de vento e renomeia colunas com prefixo.
    
    Args:
        file_path: Caminho para o arquivo
        prefix: Prefixo para as colunas (Dir, W, WS)
    
    Returns:
        DataFrame com colunas renomeadas
    """
    df = pd.read_csv(file_path, sep=';', encoding='utf-8')
    
    # Converter coluna DT para datetime
    df['DT'] = pd.to_datetime(df['DT'])
    
    # Renomear colunas (exceto DT) com o prefixo
    column_mapping = {col: f"{prefix}-{col}" for col in df.columns if col != 'DT'}
    df = df.rename(columns=column_mapping)
    
    print(f"Arquivo {file_path.name} carregado: {len(df)} registros")
    print(f"Período: {df['DT'].min()} até {df['DT'].max()}")
    
    return df

print("Função de leitura definida")

Função de leitura definida


In [4]:
# Carregar todos os arquivos
dataframes = []

for file_name in files_to_process:
    file_path = raw_data_path / file_name
    prefix = file_name.replace('.txt', '')  # Remove extensão para usar como prefixo
    
    if file_path.exists():
        df = read_wind_data(file_path, prefix)
        dataframes.append(df)
        print(f"✓ {file_name} processado")
    else:
        print(f"✗ Arquivo {file_name} não encontrado")

print(f"\nTotal de arquivos carregados: {len(dataframes)}")

Arquivo Dir.txt carregado: 41786 registros
Período: 2021-11-10 17:20:00 até 2022-10-09 08:30:00
✓ Dir.txt processado
Arquivo W.txt carregado: 41786 registros
Período: 2021-11-10 17:20:00 até 2022-10-09 08:30:00
✓ W.txt processado
Arquivo WS.txt carregado: 41786 registros
Período: 2021-11-10 17:20:00 até 2022-10-09 08:30:00
✓ WS.txt processado

Total de arquivos carregados: 3


In [5]:
# Combinar todos os DataFrames usando merge na coluna DT
if len(dataframes) > 0:
    # Começar com o primeiro DataFrame
    combined_df = dataframes[0]
    
    # Fazer merge com os demais
    for df in dataframes[1:]:
        combined_df = pd.merge(combined_df, df, on='DT', how='outer')
    
    print(f"Dataset combinado criado com {len(combined_df)} registros")
    print(f"Colunas no dataset: {len(combined_df.columns)}")
    print(f"Primeiras colunas: {list(combined_df.columns[:10])}")
else:
    print("Nenhum arquivo foi carregado")

Dataset combinado criado com 42524 registros
Colunas no dataset: 79
Primeiras colunas: ['DT', 'Dir-SODAR-30', 'Dir-SODAR-40', 'Dir-SODAR-50', 'Dir-SODAR-60', 'Dir-SODAR-70', 'Dir-SODAR-80', 'Dir-SODAR-90', 'Dir-SODAR-100', 'Dir-SODAR-110']


In [6]:
# Filtrar apenas as colunas de SODAR-40 até SODAR-140
# Primeiro renomear DT para timestamp
combined_df = combined_df.rename(columns={'DT': 'timestamp'})

# Selecionar colunas timestamp e SODAR-40 até SODAR-140
sodar_columns = ['timestamp']
for height in range(40, 150, 10):  # 40, 50, 60, ..., 140
    for prefix in ['Dir', 'W', 'WS']:
        col_name = f"{prefix}-SODAR-{height}"
        if col_name in combined_df.columns:
            sodar_columns.append(col_name)

# Criar DataFrame filtrado
wind_data = combined_df[sodar_columns].copy()

print(f"Dataset filtrado criado com {len(wind_data)} registros")
print(f"Colunas selecionadas: {len(wind_data.columns) - 1} (excluindo timestamp)")
print(f"Alturas SODAR incluídas: 40m até 140m")

Dataset filtrado criado com 42524 registros
Colunas selecionadas: 33 (excluindo timestamp)
Alturas SODAR incluídas: 40m até 140m


In [7]:
# Verificar estrutura atual dos dados
print("=== Estrutura do Dataset Atual ===")
print(f"Forma: {wind_data.shape}")
print(f"\nPrimeiros e últimos timestamps:")
print(f"Início: {wind_data['timestamp'].min()}")
print(f"Fim: {wind_data['timestamp'].max()}")
print(f"\nVerificação de valores nulos por coluna:")
print(wind_data.isnull().sum().head(10))

=== Estrutura do Dataset Atual ===
Forma: (42524, 34)

Primeiros e últimos timestamps:
Início: 2021-11-10 17:20:00
Fim: 2022-10-09 08:30:00

Verificação de valores nulos por coluna:
timestamp         0
Dir-SODAR-40    487
W-SODAR-40      434
WS-SODAR-40     487
Dir-SODAR-50    474
W-SODAR-50      489
WS-SODAR-50     474
Dir-SODAR-60    486
W-SODAR-60      536
WS-SODAR-60     486
dtype: int64


In [8]:
# Regularizar série temporal - criar índice completo com intervalos de 10 minutos
print("=== Regularização da Série Temporal ===")

# Ordenar por timestamp
wind_data = wind_data.sort_values('timestamp')

# Remover duplicatas de timestamp
duplicates_before = wind_data.duplicated(subset=['timestamp']).sum()
wind_data = wind_data.drop_duplicates(subset=['timestamp'])
print(f"Duplicatas removidas: {duplicates_before}")

# Criar série temporal completa com intervalos de 10 minutos
start_time = wind_data['timestamp'].min()
end_time = wind_data['timestamp'].max()

# Gerar todos os timestamps de 10 em 10 minutos
complete_timeline = pd.date_range(start=start_time, end=end_time, freq='10min')
complete_df = pd.DataFrame({'timestamp': complete_timeline})

print(f"Período original: {start_time} até {end_time}")
print(f"Registros originais: {len(wind_data)}")
print(f"Timestamps esperados (10min): {len(complete_timeline)}")
print(f"Timestamps faltantes: {len(complete_timeline) - len(wind_data)}")

=== Regularização da Série Temporal ===
Duplicatas removidas: 833
Período original: 2021-11-10 17:20:00 até 2022-10-09 08:30:00
Registros originais: 41691
Timestamps esperados (10min): 47900
Timestamps faltantes: 6209


In [9]:
# Fazer merge para preencher timestamps faltantes com NaN
wind_data_regularized = pd.merge(complete_df, wind_data, on='timestamp', how='left')

print("=== Dataset Regularizado ===")
print(f"Forma final: {wind_data_regularized.shape}")
print(f"Total de timestamps: {len(wind_data_regularized)}")
print(f"Timestamps únicos: {wind_data_regularized['timestamp'].nunique()}")

# Verificar se há duplicatas finais
final_duplicates = wind_data_regularized.duplicated(subset=['timestamp']).sum()
print(f"Duplicatas finais: {final_duplicates}")

# Estatísticas de valores faltantes
missing_stats = wind_data_regularized.isnull().sum()
print(f"\nEstatísticas de valores faltantes:")
print(f"Colunas com dados faltantes: {(missing_stats > 0).sum()}")
print(f"Total de valores NaN: {missing_stats.sum()}")

=== Dataset Regularizado ===
Forma final: (47900, 34)
Total de timestamps: 47900
Timestamps únicos: 47900
Duplicatas finais: 0

Estatísticas de valores faltantes:
Colunas com dados faltantes: 33
Total de valores NaN: 233444


In [10]:
# Exibir amostra dos dados finais
print("=== Amostra do Dataset Final ===")
print("Primeiras 5 linhas:")
print(wind_data_regularized.head())
print("\nÚltimas 5 linhas:")
print(wind_data_regularized.tail())
print("\nInfo do dataset:")
print(wind_data_regularized.info())

=== Amostra do Dataset Final ===
Primeiras 5 linhas:
            timestamp  Dir-SODAR-40  W-SODAR-40  WS-SODAR-40  Dir-SODAR-50  \
0 2021-11-10 17:20:00          29.7        0.08         5.03          30.5   
1 2021-11-10 17:30:00          38.1       -0.08         4.94          37.7   
2 2021-11-10 17:40:00          41.4        0.02         4.44          42.7   
3 2021-11-10 17:50:00          36.2        0.04         4.28          44.3   
4 2021-11-10 18:00:00          58.9       -0.04         4.53          49.5   

   W-SODAR-50  WS-SODAR-50  Dir-SODAR-60  W-SODAR-60  WS-SODAR-60  ...  \
0        0.03         5.53          29.2       -0.03         5.54  ...   
1       -0.05         5.30          35.0       -0.13         5.29  ...   
2       -0.07         5.96          41.3       -0.12         5.66  ...   
3        0.04         4.91          45.7        0.00         5.04  ...   
4       -0.09         5.02          44.6       -0.13         4.66  ...   

   WS-SODAR-110  Dir-SODAR-120  W

In [11]:
# Salvar o dataset regularizado
output_file = output_path / 'wind_data.csv'

# Salvar o arquivo
wind_data_regularized.to_csv(output_file, index=False, encoding='utf-8')

print(f"=== Dataset Salvo ===")
print(f"Arquivo: {output_file}")
print(f"Tamanho: {len(wind_data_regularized)} registros")
print(f"Colunas: {len(wind_data_regularized.columns)}")
print(f"Período: {wind_data_regularized['timestamp'].min()} até {wind_data_regularized['timestamp'].max()}")

# Verificar se arquivo foi criado
if output_file.exists():
    file_size = output_file.stat().st_size / (1024 * 1024)  # MB
    print(f"✓ Arquivo criado com sucesso ({file_size:.2f} MB)")
else:
    print("✗ Erro ao criar arquivo")

=== Dataset Salvo ===
Arquivo: ../wind_data.csv
Tamanho: 47900 registros
Colunas: 34
Período: 2021-11-10 17:20:00 até 2022-10-09 08:30:00
✓ Arquivo criado com sucesso (7.75 MB)


In [12]:
# Resumo final do processamento
print("=== RESUMO DO PROCESSAMENTO ===")
print(f"\n1. Arquivos processados: {', '.join(files_to_process)}")
print(f"2. Dataset combinado: wind_data.csv")
print(f"3. Período de dados: {wind_data_regularized['timestamp'].min()} até {wind_data_regularized['timestamp'].max()}")
print(f"4. Intervalo: 10 minutos")
print(f"5. Total de registros: {len(wind_data_regularized):,}")
print(f"6. Colunas de dados: {len(wind_data_regularized.columns) - 1} (SODAR-40 até SODAR-140)")
print(f"7. Duplicatas removidas: {duplicates_before}")
print(f"8. Timestamps adicionados: {len(complete_timeline) - len(wind_data) + duplicates_before}")
print(f"\n✓ Processamento concluído com sucesso!")

=== RESUMO DO PROCESSAMENTO ===

1. Arquivos processados: Dir.txt, W.txt, WS.txt
2. Dataset combinado: wind_data.csv
3. Período de dados: 2021-11-10 17:20:00 até 2022-10-09 08:30:00
4. Intervalo: 10 minutos
5. Total de registros: 47,900
6. Colunas de dados: 33 (SODAR-40 até SODAR-140)
7. Duplicatas removidas: 833
8. Timestamps adicionados: 7042

✓ Processamento concluído com sucesso!
